# First Version



In [58]:
# CMPT459 2020 spring
# milestone2
# data cleaned and feature selection and numericalization for classifier
# group name: Salt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv as csv
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split

def outlier_fd(data, threshold=0.5):
    low, high = np.percentile(data,1), np.percentile(data, 99)
    dist = high - low
    cut_off = dist * threshold
    lower, upper = low-cut_off, high+cut_off
    return [lower, upper]

def photos_number_counter(x):
    return len(x)

def valid_des(s):
    valid_s = " "
    for i in s.split():
        if i.isalnum():
            valid_s = " ".join([valid_s, i])
            
    return valid_s.strip()



In [59]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
ptrain = train
ptest = test
ptrain = ptrain.drop(['interest_level'], axis = 1)
pwhole = ptrain.append(ptest)

# Data Prepocessing for training data and test data

In [60]:
# Find the outlier of price, latitude and longitude using bound given by outlier_fd
[price_lower, price_higher] = outlier_fd(ptrain['price'])
[latitude_lower, latitude_higher] = outlier_fd(ptrain['latitude'],1)
[longitude_lower, longitude_higher] = outlier_fd(ptrain['longitude'],1)
#deal with outliers, set the column for modified data and plot again
pwhole['price_modify'] = pwhole['price']
pwhole['price_modify'].loc[pwhole['price_modify']<price_lower] = price_lower
pwhole['price_modify'].loc[pwhole['price_modify']>price_higher] = price_higher
#deal with outliers, set the column for modified data and plot again
pwhole['latitude_modify'] = pwhole['latitude']
pwhole['latitude_modify'].loc[pwhole['latitude_modify']<latitude_lower] = latitude_lower
pwhole['latitude_modify'].loc[pwhole['latitude_modify']>latitude_higher] = latitude_higher
#deal with outliers, set the column for modified data and plot again
pwhole['longitude_modify'] = pwhole['longitude']
pwhole['longitude_modify'].loc[pwhole['longitude']<longitude_lower] = longitude_lower
pwhole['longitude_modify'].loc[pwhole['longitude']>longitude_higher] = longitude_higher
#get the number of photos
pwhole['photos_number'] = pwhole['photos'].apply(photos_number_counter)
#set all feature words in tf_idf as features for train for feature_modify
pwhole['features_modify'] = pwhole['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
counterf = CountVectorizer(max_features=100)
train_sparse_f = counterf.fit_transform(pwhole['features_modify'])
feature_list = counterf.get_feature_names()
array1 = train_sparse_f.toarray()
for i, feature_str in enumerate(feature_list):
    list_all = []
    [rows, cols] = array1.shape
    for row in range(rows):
        list_all.append(array1[row][i])
    feature_str_ = feature_str + '(f)'
    feature_v = pd.Series(list_all,pwhole.index, name = feature_str_)
    pwhole[feature_str_] = feature_v
#set all feature words in tf_idf as features for train for description_modify
pwhole['description_modify'] = pwhole['description'].apply(lambda x:valid_des(x))
counterd = CountVectorizer(stop_words=['features'], max_features=100)
train_sparse_d = counterd.fit_transform(pwhole['description_modify'])
description_list = counterd.get_feature_names()
array2 = train_sparse_d.toarray()
for i, description_str in enumerate(description_list):
    list_all = []
    [rows, cols] = array2.shape
    for row in range(rows):
        list_all.append(array2[row][i])
    description_str_ = description_str + '(d)'
    description_v = pd.Series(list_all,pwhole.index, name = description_str_)
    pwhole[description_str_] = description_v
## /*add something to change the created to year month day*/
pwhole['created']=pd.to_datetime(pwhole["created"])
pwhole['year']=pwhole['created'].dt.year
pwhole['month']=pwhole['created'].dt.month
pwhole['day']=pwhole['created'].dt.day



/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Feature Selection

In [61]:
pwhole = pwhole.drop(['central_a(f)','in_super(f)','on(f)','pre(f)','war(f)'], axis = 1)
pwhole = pwhole.drop(['an(d)','and(d)','are(d)','as(d)','at(d)','be(d)','by(d)','can(d)',
                      'for(d)','from(d)','in(d)','is(d)','it(d)','just(d)','me(d)','more(d)',
                      'natural(d)','new(d)','or(d)','park(d)','that(d)','the(d)','this(d)',
                      'to(d)','will(d)','with(d)','you(d)','your(d)'], axis = 1)
pwhole = pwhole.drop(["building_id", "created", "description", "display_address", "features", 
                     "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify"], axis=1)


In [62]:
X = pwhole[:49352]
X_Test = pwhole[49352:]
X = X.drop(['listing_id'], axis = 1)
X_Test = X_Test.drop(['listing_id'], axis = 1)
Y = pd.get_dummies(train['interest_level'])


Now we have three datasets:

training samples(without labels) -- X

training labels -- Y

test dataset -- X_Test


the columns of X_train is shown below:

In [63]:
Y

,high,low,medium
4,0,0,1
6,0,1,0
9,0,0,1
10,0,0,1
15,0,1,0
...,...,...,...
124000,0,1,0
124002,0,0,1
124004,0,0,1
124008,0,0,1


In [64]:
# for i in range(0,49352):
#     if Y.loc[i,'high']==1:
#         Y.loc[i, 'target']=0
#     if Y.loc[i,'low']==1:
#         Y.loc[i, 'target']=1
#     if Y.loc[i,'medium']==1:
#         Y.loc[i, 'target']=2
# Y=Y.drop(['high','low','medium'], axis = 1)
# Y

# 5-fold Cross-validation (repeats 5 times)

I use 5-fold cross-validation here and choose the classifer with lowest log loss(Best_OVR1), highest classification accuracy(Best_OVR2), highest precision(Best_OVR3)

In [65]:
training_score = 0
validation_score = 0
Best_validation_score = 20
count = 0
total_validation_score = 0
precision = []
recall = []
test_accuracy =[]
test_accuracy_value = 0
test_precision = 0

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
# from sklearn.model_selection import cross_val_score
# scrores
#maybe 5-fold is better than 10-fold
kf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=None) 

for train_index, test_index in kf.split(X):
#     print("Train:", train_index, "Validation:",test_index)
    X['id']=list(range(0,49352))
    X.set_index("id", inplace=True)
    Y['id']=list(range(0,49352))
    Y.set_index("id", inplace=True)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    Y_train, Y_test = Y.loc[train_index,:], Y.loc[test_index,:]
#     OVR = OneVsRestClassifier(LogisticRegression(warm_start=True,max_iter=200,tol=0.00001)).fit(X_train,Y_train)
#     ABC = AdaBoostClassifier(n_estimators=150,base_estimator=LogisticRegression(warm_start=True,max_iter=200,tol=0.00001))
#     rfc = RandomForestClassifier()
#     rfc.fit(X_train,Y_train)
#     ABC = AdaBoostClassifier(n_estimators=5,base_estimator=LogisticRegression(warm_start=True,max_iter=200,tol=0.00001))
#     ABC = OneVsRestClassifier(AdaBoostClassifier(n_estimators=50,base_estimator=DecisionTreeClassifier(max_depth=6)))

#     ABC = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=150))
    ABC = OneVsRestClassifier(HistGradientBoostingClassifier(max_iter=50, tol=1e-07))
    ABC = ABC.fit(X_train,Y_train)
#   classification accuracy metrics
    if test_accuracy_value<ABC.score(X_test,Y_test):
        test_accuracy_value=ABC.score(X_test,Y_test)
        Best_ABC2 = ABC
    test_accuracy.append(ABC.score(X_test,Y_test))
#   log loss on training dataset
    Y_Prediction=ABC.predict_proba(X_train)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_train=pd.DataFrame(Y_train)
    training_score=log_loss(Y_train, Y_Prediction)
    print("training log_loss: %0.3f" % training_score)

#   log loss on test dataset
    Y_Prediction=ABC.predict_proba(X_test)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_test=pd.DataFrame(Y_test)
    validation_score=log_loss(Y_test, Y_Prediction)
    count+=1
    total_validation_score+=validation_score
    print("validation log_loss: %0.3f" % validation_score)

#   logloss metrics
    if validation_score<Best_validation_score:
        Best_validation_score = validation_score
        Best_ABC1 = ABC

    if count==5:
        average = total_validation_score/float(count)
        print("average validation log_loss: %0.3f" % average)
        count=0
        total_validation_score=0
    
#     Y_Prediction=OVR.predict(X_test)
# #   print confusion matrix
#     print(multilabel_confusion_matrix(Y_test, Y_Prediction))
# #   confusion matrix matrics
#     if test_precision<precision_score(Y_test, Y_Prediction, average='macro'):
#         test_precision=precision_score(Y_test, Y_Prediction, average='macro')
#         Best_OVR3 = OVR
#     precision.append(precision_score(Y_test, Y_Prediction, average='macro'))
#     recall.append(recall_score(Y_test, Y_Prediction, average='macro'))
print ("One vs rest best validation accuracy: %0.3f" % Best_validation_score) 
# print(precision)
# print(recall)
print (test_accuracy)

training log_loss: 0.563
validation log_loss: 0.608
training log_loss: 0.565
validation log_loss: 0.599
training log_loss: 0.565
validation log_loss: 0.598
training log_loss: 0.561
validation log_loss: 0.613
training log_loss: 0.563
validation log_loss: 0.607
average validation log_loss: 0.605
training log_loss: 0.563
validation log_loss: 0.605
training log_loss: 0.567
validation log_loss: 0.591
training log_loss: 0.559
validation log_loss: 0.621
training log_loss: 0.562
validation log_loss: 0.612
training log_loss: 0.565
validation log_loss: 0.596
average validation log_loss: 0.605
training log_loss: 0.563
validation log_loss: 0.606
training log_loss: 0.561
validation log_loss: 0.614
training log_loss: 0.565
validation log_loss: 0.597
training log_loss: 0.563
validation log_loss: 0.606
training log_loss: 0.563
validation log_loss: 0.604
average validation log_loss: 0.605
training log_loss: 0.565
validation log_loss: 0.598
training log_loss: 0.563
validation log_loss: 0.606
training lo

# Write into submission.csv

Here I use the best classifier evaluated by log loss

In [47]:
Y_Prediction=Best_ABC1.predict_proba(X_Test)  #use the best classifier evaluated by log loss
y=pd.DataFrame(Y_Prediction)
y.reset_index(drop=True, inplace=True)
ids = test['listing_id']
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, y],axis=1, ignore_index=True)
out
# out.columns=['listing_id','high','medium','low']
out.columns=['listing_id','high','low','medium']
submission_file = open("submission.csv", "w")
out.to_csv('submission.csv', index = False)
submission_file.close()

In [ ]:
Y_Prediction

In [ ]:
Y

# Final Version

Here is mainly first version and I did some feature selections corresponding to the modification part(a) in the report.

In [66]:
# CMPT459 2020 spring
# milestone2
# data cleaned and feature selection and numericalization for classifier
# group name: Salt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv as csv
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split

def outlier_fd(data, threshold=0.5):
    low, high = np.percentile(data,1), np.percentile(data, 99)
    dist = high - low
    cut_off = dist * threshold
    lower, upper = low-cut_off, high+cut_off
    return [lower, upper]

def photos_number_counter(x):
    return len(x)

def valid_des(s):
    valid_s = " "
    for i in s.split():
        if i.isalnum():
            valid_s = " ".join([valid_s, i])
            
    return valid_s.strip()




In [67]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
ptrain = train
ptest = test
ptrain = ptrain.drop(['interest_level'], axis = 1)
pwhole = ptrain.append(ptest)

In [68]:
# Find the outlier of price, latitude and longitude using bound given by outlier_fd
[price_lower, price_higher] = outlier_fd(ptrain['price'])
[latitude_lower, latitude_higher] = outlier_fd(ptrain['latitude'],1)
[longitude_lower, longitude_higher] = outlier_fd(ptrain['longitude'],1)
#deal with outliers, set the column for modified data and plot again
pwhole['price_modify'] = pwhole['price']
pwhole['price_modify'].loc[pwhole['price_modify']<price_lower] = price_lower
pwhole['price_modify'].loc[pwhole['price_modify']>price_higher] = price_higher
#deal with outliers, set the column for modified data and plot again
pwhole['latitude_modify'] = pwhole['latitude']
pwhole['latitude_modify'].loc[pwhole['latitude_modify']<latitude_lower] = latitude_lower
pwhole['latitude_modify'].loc[pwhole['latitude_modify']>latitude_higher] = latitude_higher
#deal with outliers, set the column for modified data and plot again
pwhole['longitude_modify'] = pwhole['longitude']
pwhole['longitude_modify'].loc[pwhole['longitude']<longitude_lower] = longitude_lower
pwhole['longitude_modify'].loc[pwhole['longitude']>longitude_higher] = longitude_higher
#get the number of photos
pwhole['photos_number'] = pwhole['photos'].apply(photos_number_counter)
#set all feature words in tf_idf as features for train for feature_modify
pwhole['features_modify'] = pwhole['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
counterf = CountVectorizer(max_features=100)
train_sparse_f = counterf.fit_transform(pwhole['features_modify'])
feature_list = counterf.get_feature_names()
array1 = train_sparse_f.toarray()
for i, feature_str in enumerate(feature_list):
    list_all = []
    [rows, cols] = array1.shape
    for row in range(rows):
        list_all.append(array1[row][i])
    feature_str_ = feature_str + '(f)'
    feature_v = pd.Series(list_all,pwhole.index, name = feature_str_)
    pwhole[feature_str_] = feature_v
#set all feature words in tf_idf as features for train for description_modify
pwhole['description_modify'] = pwhole['description'].apply(lambda x:valid_des(x))
counterd = CountVectorizer(stop_words=['features'], max_features=100)
train_sparse_d = counterd.fit_transform(pwhole['description_modify'])
description_list = counterd.get_feature_names()
array2 = train_sparse_d.toarray()
for i, description_str in enumerate(description_list):
    list_all = []
    [rows, cols] = array2.shape
    for row in range(rows):
        list_all.append(array2[row][i])
    description_str_ = description_str + '(d)'
    description_v = pd.Series(list_all,pwhole.index, name = description_str_)
    pwhole[description_str_] = description_v
## /*add something to change the created to year month day*/
pwhole['created']=pd.to_datetime(pwhole["created"])
pwhole['year']=pwhole['created'].dt.year
pwhole['month']=pwhole['created'].dt.month
pwhole['day']=pwhole['created'].dt.day




/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [69]:
pwhole = pwhole.drop(['central_a(f)','in_super(f)','on(f)','pre(f)','war(f)'], axis = 1)
pwhole = pwhole.drop(['an(d)','and(d)','are(d)','as(d)','at(d)','be(d)','by(d)','can(d)',
                      'for(d)','from(d)','in(d)','is(d)','it(d)','just(d)','me(d)','more(d)',
                      'natural(d)','new(d)','or(d)','park(d)','that(d)','the(d)','this(d)',
                      'to(d)','will(d)','with(d)','you(d)','your(d)'], axis = 1)
pwhole = pwhole.drop(["building_id", "created", "description", "display_address", "features", 
                     "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify"], axis=1)



In [70]:
X = pwhole[:49352]
X_Test = pwhole[49352:]
X = X.drop(['listing_id'], axis = 1)
X_Test = X_Test.drop(['listing_id'], axis = 1)
Y = pd.get_dummies(train['interest_level'])



# Feature Selection

In [71]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
fsclf = ExtraTreesClassifier()
fsclf = fsclf.fit(X, Y)
fsclf.feature_importances_

/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([9.82902159e-03, 2.35845076e-02, 5.30274181e-02, 2.96596259e-02,
       2.90466952e-02, 2.91141293e-02, 4.32790683e-04, 5.13846006e-04,
       3.30924876e-03, 1.23402200e-04, 6.36228429e-03, 5.40885101e-05,
       9.90911154e-05, 2.62655144e-03, 1.28285735e-04, 1.30855560e-04,
       1.97011745e-04, 5.13207432e-04, 4.71072767e-03, 7.61638335e-03,
       6.35055291e-03, 9.28850620e-03, 3.80912531e-04, 3.62260892e-04,
       2.36093155e-04, 1.54924129e-04, 8.50298219e-03, 4.37500326e-03,
       3.87451902e-04, 1.79251156e-03, 5.59023895e-05, 5.81537908e-03,
       4.08216663e-05, 3.14992338e-03, 7.78208351e-04, 2.34443470e-03,
       2.40457311e-04, 3.57621347e-04, 2.18632656e-04, 5.53391421e-04,
       1.40971658e-02, 4.97433845e-04, 8.26995140e-04, 3.27722169e-03,
       3.51688217e-05, 3.47158731e-04, 2.70175730e-04, 1.08309055e-02,
       7.25225888e-03, 8.27835850e-04, 9.13418984e-04, 6.42651600e-04,
       4.46281247e-04, 4.59444575e-04, 3.43506192e-03, 8.31765532e-05,
      

In [72]:
names = X.columns
names

Index(['bathrooms', 'bedrooms', 'price_modify', 'latitude_modify',
       'longitude_modify', 'photos_number', '_photos(f)', 'actual_apt(f)',
       'balcony(f)', 'bike_room(f)',
       ...
       'supports(d)', 'text(d)', 'two(d)', 'unit(d)', 'west(d)', 'windows(d)',
       'york(d)', 'year', 'month', 'day'],
      dtype='object', length=176)

In [73]:
sorted_features = sorted(zip(fsclf.feature_importances_, names), reverse=True)
sorted_features

[(0.07205740299664692, 'day'),
 (0.05302741805273241, 'price_modify'),
 (0.039770597568169606, 'month'),
 (0.029659625894016356, 'latitude_modify'),
 (0.029114129335232948, 'photos_number'),
 (0.029046695247906568, 'longitude_modify'),
 (0.023584507618324705, 'bedrooms'),
 (0.01409716582245891, 'hardwood_floors(f)'),
 (0.013801476917525191, 'apartment(d)'),
 (0.013465676884601582, 'of(d)'),
 (0.012470066082610245, 'bedroom(d)'),
 (0.012243415726651764, 'no_fee(f)'),
 (0.011946094162706573, 'on(d)'),
 (0.010830905534187226, 'laundry_in_building(f)'),
 (0.010823898278304604, 'large(d)'),
 (0.010736975760043491, 'kitchen(d)'),
 (0.010541852501813411, 'renovated(d)'),
 (0.010492825201911487, 'great(d)'),
 (0.010252643288963508, 'building(d)'),
 (0.010159821464455153, 'call(d)'),
 (0.00997143141068902, 'located(d)'),
 (0.009829021591743469, 'bathrooms'),
 (0.009672578963100044, 'hardwood(d)'),
 (0.009624788215813326, 'all(d)'),
 (0.009288506199483227, 'doorman(f)'),
 (0.009192208964599371, 

In [74]:
feature_selected = [sorted_features[i][1] for i in range(150)]
feature_selected

['day',
 'price_modify',
 'month',
 'latitude_modify',
 'photos_number',
 'longitude_modify',
 'bedrooms',
 'hardwood_floors(f)',
 'apartment(d)',
 'of(d)',
 'bedroom(d)',
 'no_fee(f)',
 'on(d)',
 'laundry_in_building(f)',
 'large(d)',
 'kitchen(d)',
 'renovated(d)',
 'great(d)',
 'building(d)',
 'call(d)',
 'located(d)',
 'bathrooms',
 'hardwood(d)',
 'all(d)',
 'doorman(f)',
 'high(d)',
 'living(d)',
 'contact(d)',
 'one(d)',
 'steel(d)',
 'beautiful(d)',
 'no(d)',
 'has(d)',
 'full(d)',
 'east(d)',
 'elevator(f)',
 'spacious(d)',
 'laundry(d)',
 'stainless(d)',
 'email(d)',
 'closet(d)',
 'room(d)',
 'unit(d)',
 'studio(d)',
 'private(d)',
 'close(d)',
 'dishwasher(f)',
 'marble(d)',
 'granite(d)',
 'size(d)',
 'bathroom(d)',
 'space(d)',
 'text(d)',
 'amazing(d)',
 'laundry_in_unit(f)',
 'appliances(d)',
 'available(d)',
 'steps(d)',
 'windows(d)',
 'two(d)',
 'floors(d)',
 'subway(d)',
 'west(d)',
 'schedule(d)',
 'heart(d)',
 'floor(d)',
 'have(d)',
 'central(d)',
 'cats_allowed(

In [75]:
X=X[feature_selected]

# Parameters Selection

In [94]:
# GridSearchCV to find optimal max_iter, tol
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5


# instantiate the model (note we are specifying a max_depth)
model_to_set = OneVsRestClassifier(HistGradientBoostingClassifier())
# parameters to build the model on
parameters = {
    "estimator__max_iter":range(210,260,10),
#     "estimator__max_leaf_nodes":range(25,40,5),
#     "estimator__tol":[1e-6,0.5*1e-7,1e-7,0.5*1e-8,1e-8],
#     "estimator__min_samples_leaf":range(10,25,5)
#     "estimator__max_depth":[5,6,7]    
}
# fit tree on training data
HGB = GridSearchCV(model_to_set, parameters, 
                    cv=n_folds, 
                   scoring="neg_log_loss")   #neg_log_loss is simply equal to - log_loss
HGB.fit(X, Y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=OneVsRestClassifier(estimator=HistGradientBoostingClassifier(l2_regularization=0.0,
                                                                                    learning_rate=0.1,
                                                                                    loss='auto',
                                                                                    max_bins=256,
                                                                                    max_depth=None,
                                                                                    max_iter=100,
                                                                                    max_leaf_nodes=31,
                                                                                    min_samples_leaf=20,
                                                                                    n_iter_no_change=None,
                              

In [95]:
# scores of GridSearch CV
scores = HGB.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,27.002723,1.528679,0.137892,0.011177,210,{'estimator__max_iter': 210},-0.567558,-0.576933,-0.587676,-0.599680,-0.596830,-0.585735,0.012077,5
1,25.529671,0.495937,0.133028,0.023447,220,{'estimator__max_iter': 220},-0.567105,-0.576243,-0.587271,-0.599493,-0.596717,-0.585365,0.012237,4
2,25.138426,0.675040,0.129332,0.014783,230,{'estimator__max_iter': 230},-0.566790,-0.576096,-0.587312,-0.599803,-0.596266,-0.585253,0.012345,3
3,26.039024,0.174455,0.139787,0.012643,240,{'estimator__max_iter': 240},-0.566430,-0.575782,-0.587034,-0.599966,-0.596183,-0.585079,0.012514,2
4,26.550330,0.192151,0.130377,0.005729,250,{'estimator__max_iter': 250},-0.566456,-0.575458,-0.587269,-0.600047,-0.596028,-0.585051,0.012555,1


In [96]:
HGB.best_params_['estimator__max_iter']

250

In [97]:
parameters = {
    "estimator__tol":[1e-6,1e-7,1e-8]
}

# fit tree on training data
HGB = GridSearchCV(model_to_set, parameters, 
                    cv=n_folds, 
                   scoring="neg_log_loss")   #neg_log_loss is simply equal to - log_loss
# HGB = OneVsRestClassifier(HGB)
HGB.fit(X, Y)
HGB.best_params_['estimator__tol']

1e-06

In [100]:
training_score = 0
validation_score = 0
Best_validation_score = 20
count = 0
total_validation_score = 0
precision = []
recall = []
test_accuracy =[]
test_accuracy_value = 0
test_precision = 0

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
import lightgbm as lgb


kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=None) 


for train_index, test_index in kf.split(X):
#     print("Train:", train_index, "Validation:",test_index)
    X['id']=list(range(0,49352))
    X.set_index("id", inplace=True)
    Y['id']=list(range(0,49352))
    Y.set_index("id", inplace=True)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    Y_train, Y_test = Y.loc[train_index,:], Y.loc[test_index,:]
    ABC = OneVsRestClassifier(HistGradientBoostingClassifier(max_iter=250, tol=1e-08)) #max_iter=200 300not good
    ABC = ABC.fit(X_train,Y_train)
#   classification accuracy metrics
    if test_accuracy_value<ABC.score(X_test,Y_test):
        test_accuracy_value=ABC.score(X_test,Y_test)
        Best_ABC2 = ABC
    test_accuracy.append(ABC.score(X_test,Y_test))
#   log loss on training dataset
    Y_Prediction=ABC.predict_proba(X_train)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_train=pd.DataFrame(Y_train)
    training_score=log_loss(Y_train, Y_Prediction)
    print("training log_loss: %0.3f" % training_score)

#   log loss on test dataset
    Y_Prediction=ABC.predict_proba(X_test)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_test=pd.DataFrame(Y_test)
    validation_score=log_loss(Y_test, Y_Prediction)
    count+=1
    total_validation_score+=validation_score
    print("validation log_loss: %0.3f" % validation_score)

#   logloss metrics
    if validation_score<Best_validation_score:
        Best_validation_score = validation_score
        Best_ABC1 = ABC

    if count==10:
        average = total_validation_score/float(count)
        print("average validation log_loss: %0.3f" % average)
        count=0
        total_validation_score=0
    
    Y_Prediction=ABC.predict(X_test)
#   print confusion matrix
    print(multilabel_confusion_matrix(Y_test, Y_Prediction))
#   confusion matrix matrics
    if test_precision<precision_score(Y_test, Y_Prediction, average='macro'):
        test_precision=precision_score(Y_test, Y_Prediction, average='macro')
        Best_ABC3 = ABC
    precision.append(precision_score(Y_test, Y_Prediction, average='macro'))
    recall.append(recall_score(Y_test, Y_Prediction, average='macro'))
print ("One vs rest best validation accuracy: %0.3f" % Best_validation_score) 
print(precision)
print(recall)
print (test_accuracy)

training log_loss: 0.436
validation log_loss: 0.583
[[[4509   54]
  [ 279   94]]

 [[ 884  622]
  [ 354 3076]]

 [[3651  152]
  [ 970  163]]]
training log_loss: 0.433
validation log_loss: 0.602
[[[4439   61]
  [ 334  102]]

 [[ 911  659]
  [ 356 3010]]

 [[3632  170]
  [ 959  175]]]
training log_loss: 0.437
validation log_loss: 0.566
[[[4520   45]
  [ 290   80]]

 [[ 852  612]
  [ 333 3138]]

 [[3694  147]
  [ 937  157]]]
training log_loss: 0.434
validation log_loss: 0.588
[[[4478   44]
  [ 312  101]]

 [[ 876  659]
  [ 380 3020]]

 [[3650  163]
  [ 974  148]]]
training log_loss: 0.437
validation log_loss: 0.562
[[[4543   48]
  [ 268   76]]

 [[ 826  623]
  [ 363 3123]]

 [[3689  141]
  [ 958  147]]]
training log_loss: 0.437
validation log_loss: 0.590
[[[4489   43]
  [ 315   88]]

 [[ 864  630]
  [ 343 3098]]

 [[3697  147]
  [ 945  146]]]
training log_loss: 0.436
validation log_loss: 0.581
[[[4544   47]
  [ 268   76]]

 [[ 862  662]
  [ 357 3054]]

 [[3596  159]
  [1015  165]]]
traini

In [101]:
X_Test=X_Test[feature_selected]
Y_Prediction=Best_ABC1.predict_proba(X_Test)  #use the best classifier evaluated by log loss
y=pd.DataFrame(Y_Prediction)
y.reset_index(drop=True, inplace=True)
ids = test['listing_id']
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, y],axis=1, ignore_index=True)
out
# out.columns=['listing_id','high','medium','low']
out.columns=['listing_id','high','low','medium']
submission_file = open("submission.csv", "w")
out.to_csv('submission.csv', index = False)
submission_file.close()